# <div class="h1">iWildCam 2020 - FGVC7</div>

## Let's detect wild animals in new places!

### <u>Note</u>

**Update information**

Update1(Commit5): I added visualization of data distribution.

Update2(Commit9): I organized graphs for easy comparison of datas, and add comments.

Update3(Commit15): Added analysis of populations appearing in photos. And edit appendix.

Update4(Commit19): Added EfficientNet-B3 example. 

Update5(Commit21): Added analysis of timelly and locationaly apparence in animals.

In [ ]:
%%HTML
<style type="text/css">
div.h1 {
    background-color:#e17b34; 
    color: white; 
    padding: 8px; 
    padding-right: 300px; 
    font-size: 35px; 
    max-width: 1500px; 
    margin: auto; 
    margin-top: 50px;
}

div.h2 {
    background-color:#83ccd2; 
    color: white; 
    padding: 8px; 
    padding-right: 300px; 
    font-size: 35px; 
    max-width: 1500px; 
    margin: auto; 
    margin-top: 50px;
}
</style>

## About this kernel

I want to give you an overview of the prior knowledge and data that might be needed in this challenge!

This contest aims to detect wildlife in trapped picture at new monitoring locations. Now, we can get great insights about wildlife by camera traps. Camera trap is popular method and there are so many data in the world. But due to the so large number of data, it seems that the data was not always effectively accessed and utilized. 

In this competition, we aim to develop a model that effectively classifies animals taken at different observation points　in the world. This challenge. This challenge will surely bring great insights. 

### We can deal great wildlife picures!

### Let's take a look!!!😺

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

fig = plt.figure(figsize=(20, 12))
im = Image.open('../input/iwildcam-2020-fgvc7/train/939e25fc-21bc-11ea-a13a-137349068a90.jpg')
plt.imshow(im)

How cute!😻

# <div class="h2">knowladge</div>

## How can we take wildlife pictures and use?

We can take wildlife pictures by camera trap. Camera traps have infrared sensor or motion sensor, so they can detect animals. When animals come near, camera take their pictures.  Using camera traps, we can monitor wildlifes continuously ,at several point and  at the same time. So we can understand how animals run their life in the area researchers interest.[1]

For example, in Kaen Krachan National Park in Thailand, indian sinatra are directlly observed. So it was pointed out that there is no longer any possibility. But by using camera trap, we could confirm thir existence. [2]  


Traditionally, camera traps have been considered an excellent method for investigating ecological information about wildlife in a certain area.　Data are used for population estimation, calculating population index, 24hours monitorling and so on.[3]

Resently there has been a movement to make effective use of photos from camera traps using machine learning. One of the example is [4]. Google successed to access so much wildlife photo knowledge. Following Wilflife Insights is the platform of taking the initiative.

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('qKgRbkCkRFY')

Thus, creating a model for effectively classifying wild animals is a very significant effort.

### Referece

[1]https://www.wwf.org.uk/project/conservationtechnology/camera-trap

[2]https://www.wwf.or.jp/campaign/2015_camera/

[3]https://en.wikipedia.org/wiki/Camera_trap

[4]https://www.blog.google/products/earth/ai-finds-where-the-wild-things-are/

# <div class="h2">Data Overview</div>

In [ ]:
from PIL import Image, ImageDraw
import collections
import glob 
from datetime import datetime as dt
import gc
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline

## <u>Imagination of submission</u>

From https://www.kaggle.com/c/iwildcam-2020-fgvc7/overview/evaluation:
> The Id column corresponds to the test image id. The Category is an integer value that indicates the class of the animal, or 0 to represent the absence of an animal.

In [ ]:
pd.read_csv("../input/iwildcam-2020-fgvc7/sample_submission.csv").head()

## <u>Overview of train & test data</u>

There are color data and gray data.  There seems to be photos taken while it was bright and photos taken at night.

In [ ]:
train_jpeg = glob.glob('../input/iwildcam-2020-fgvc7/train/*')
test_jpeg = glob.glob('../input/iwildcam-2020-fgvc7/test/*')

print("number of train jpeg data:", len(train_jpeg))
print("number of test jpeg data:", len(test_jpeg))

### Train data

In [ ]:
fig = plt.figure(figsize=(25, 16))
for i,im_path in enumerate(train_jpeg[:16]):
    ax = fig.add_subplot(4, 4, i+1, xticks=[], yticks=[])
    im = Image.open(im_path)
    im = im.resize((480,270))
    plt.imshow(im)

In [ ]:
fig = plt.figure(figsize=(25, 16))
for i,im_path in enumerate(train_jpeg[16:32]):
    ax = fig.add_subplot(4, 4, i+1, xticks=[], yticks=[])
    im = Image.open(im_path)
    im = im.resize((480,270))
    plt.imshow(im)

### Test data

In [ ]:
fig = plt.figure(figsize=(25, 16))
for i,im_path in enumerate(test_jpeg[:16]):
    ax = fig.add_subplot(4, 4, i+1, xticks=[], yticks=[])
    im = Image.open(im_path)
    im = im.resize((480,270))
    plt.imshow(im)

## <u>Checking iwildcam2020_megadetector_results.json</u>

This json contains the detection result of the training data by megadetector.

Megadetector is the pre-trained model to detect animals, people, and vehicles in camera trap images.

https://github.com/microsoft/CameraTraps/blob/master/megadetector.md#our-ask-to-megadetector-users

In [ ]:
with open('../input/iwildcam-2020-fgvc7/iwildcam2020_megadetector_results.json', encoding='utf-8') as json_file:
    megadetector_results =json.load(json_file)
    
megadetector_results.keys()

In [ ]:
megadetector_results_df = pd.DataFrame(megadetector_results["images"])
megadetector_results_df.head()

In [ ]:
#Refered: https://www.kaggle.com/qinhui1999/how-to-use-bbox-for-iwildcam-2020 

def draw_bboxs(detections_list, im):
    """
    detections_list: list of set includes bbox.
    im: image read by Pillow.
    """
    
    for detection in detections_list:
        x1, y1,w_box, h_box = detection["bbox"]
        ymin,xmin,ymax, xmax=y1, x1, y1 + h_box, x1 + w_box
        draw = ImageDraw.Draw(im)
        
        imageWidth=im.size[0]
        imageHeight= im.size[1]
        (left, right, top, bottom) = (xmin * imageWidth, xmax * imageWidth,
                                      ymin * imageHeight, ymax * imageHeight)
        
        draw.line([(left, top), (left, bottom), (right, bottom),
               (right, top), (left, top)], width=4, fill='Red')

In [ ]:
data_num = 8857
im = Image.open("../input/iwildcam-2020-fgvc7/train/" + megadetector_results_df.loc[data_num]['id'] + ".jpg")
im = im.resize((480,270))
draw_bboxs(megadetector_results_df.loc[data_num]['detections'], im)

In [ ]:
plt.imshow(im)

We can also know how many aminals detected by megadetector in each pictures. 

In [ ]:
def get_data(x):
    if x == []:
        return 0
    return len(x)

megadetector_results_df["detected_num"] = megadetector_results_df.loc[:, "detections"].map(get_data)

In [ ]:
fig = plt.figure(figsize=(15, 4))
ax = sns.countplot(x="detected_num", data=megadetector_results_df)
ax.set(ylabel='count')
#ax.set(ylim=(0,80000))
plt.title('distribution of count per animals each data of train')

We can select training data depending on the number of animals in the picture.

In [ ]:
megadetector_results_df = megadetector_results_df[megadetector_results_df['detected_num'] < 2]
megadetector_results_df = megadetector_results_df.rename(columns={'id': 'image_id'})
megadetector_results_df.head()
#We can get filtered data of following df_train_file_cat
#pd.merge(megadetector_results_df, df_train_file_cat, on='image_id', how='inner')

## <u>Data format conversion</u>

I think, maybe you are interested in data such as category, location and datetime.

These data are provided as json data

For comvenience, I convert json data to DataFrame.

### Train data

In [ ]:
with open('/kaggle/input/iwildcam-2020-fgvc7/iwildcam2020_train_annotations.json') as json_file:
    train_annotations_json = json.load(json_file)

In [ ]:
train_annotations_json.keys()

In [ ]:
df_annotations = pd.DataFrame(train_annotations_json["annotations"])

In [ ]:
df_annotations.head()

In [ ]:
df_images = pd.DataFrame(train_annotations_json["images"])

In [ ]:
df_images.head()

In [ ]:
df_categories = pd.DataFrame(train_annotations_json["categories"])

In [ ]:
df_categories.head()

In [ ]:
print(f"There are {len(df_categories) - 1} species")

In [ ]:
# I export DataFrame as CSV for convenience. 
df_annotations.to_csv("iwildcam2020_train_annotations_annotations.csv", index=False)
df_images.to_csv("iwildcam2020_train_annotations_images.csv", index=False)
df_categories.to_csv("iwildcam2020_train_annotations_categories.csv", index=False)

### Test data

In [ ]:
with open('/kaggle/input/iwildcam-2020-fgvc7/iwildcam2020_test_information.json') as json_file:
    test_information_json = json.load(json_file)

In [ ]:
test_information_json.keys()

In [ ]:
df_images_test = pd.DataFrame(test_information_json["images"])
df_images_test.head()

In [ ]:
df_categories_test = pd.DataFrame(test_information_json["categories"])
df_categories_test.head()

In [ ]:
# I export DataFrame as CSV for convenience. 
df_images_test.to_csv("iwildcam2020_train_annotations_images_test.csv", index=False)
df_categories_test.to_csv("iwildcam2020_train_annotations_categories_test.csv", index=False)

## <u>Data distributions</u>

I plot train and test data in following perspective:

* Time point

* Category ID

* Location

### When did data taken?

Because animals can change their activity from time to time, we want to understand how data is distributed over time.

**Monthly perspective**

In [ ]:
month_year = df_images['datetime'].map(lambda str: str[2:7])
labels_month_year = sorted(list(set(month_year)))

month_year_test = df_images_test['datetime'].map(lambda str: str[2:7])

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(30,7))
ax = plt.subplot(1,2,1)
ax = plt.title('Count of train data per month & year')
ax = sns.countplot(month_year, order=labels_month_year)
ax.set(xlabel='YY-mm', ylabel='count')
ax.set(ylim=(0,50000))

ax = plt.subplot(1,2,2)
ax = plt.title('Count of test data per month & year')

ax = sns.countplot(month_year_test, order=labels_month_year)
ax.set(xlabel='YY-mm', ylabel='count')
ax.set(ylim=(0,50000))

Data starts 2013-01 but there seems be some lacks. For example, train data between 2013-11 to 2014-02 are missing. 

Also we can find that train data in between 2013-01 to 2013-07 are rich than other time point.

Train data covers test data in perspective of time point.

In [ ]:
labels_month = sorted(list(set(df_images['datetime'].map(lambda str: str[5:7]))))

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(20,7))
ax = plt.subplot(1,2,1)
plt.title('Count of train data per month')
ax = sns.countplot(df_images['datetime'].map(lambda str: str[5:7] ), order=labels_month)
ax.set(xlabel='mm', ylabel='count')
ax.set(ylim=(0,55000))

ax = plt.subplot(1,2,2)
plt.title('Count of test data per month')
ax = sns.countplot(df_images_test['datetime'].map(lambda str: str[5:7] ), order=labels_month)
ax.set(xlabel='mm', ylabel='count')
ax.set(ylim=(0,55000))

Train data are bias. In February, March, June and July, data are rich than other months. 

Train data covers test data in perspective of month.

Data for November and December are missing. Do animals hibernate?

**Hourly perspective**

In [ ]:
train_taken_hour = df_images['datetime'].map(lambda x: dt.strptime(x, '%Y-%m-%d %H:%M:%S.%f').hour)
test_taken_hour = df_images_test['datetime'].map(lambda x: dt.strptime(x, '%Y-%m-%d %H:%M:%S.%f').hour)

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(20,7))
ax = plt.subplot(1,2,1)
plt.title('Count of train data per hour')
ax = sns.countplot(train_taken_hour)
ax.set(xlabel='hour', ylabel='count')
ax.set(ylim=(0,20000))

ax = plt.subplot(1,2,2)
plt.title('Count of test data per hour')
ax = sns.countplot(test_taken_hour)
ax.set(xlabel='hour', ylabel='count')
ax.set(ylim=(0,20000))

If we decide arbitrarily during daytime and at night, we can also calculate diurnal and nocturnal data counts.

For example, we define "during daytime" is "6~17 O'clock" and "at night" is "18~5 O'clock",

In [ ]:
train_taken_phase = train_taken_hour.map(lambda x: "daytime" if x >= 6 and x < 18 else "night")
test_taken_phase = test_taken_hour.map(lambda x: "daytime" if x >= 6 and x < 18 else "night")

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(20,7))
ax = plt.subplot(1,2,1)
plt.title('Count of train data per phase')
ax = sns.countplot(train_taken_phase, order=["daytime", "night"])
ax.set(xlabel='phase', ylabel='count')
ax.set(ylim=(0,200000))

ax = plt.subplot(1,2,2)
plt.title('Count of test data per phase')
ax = sns.countplot(test_taken_phase, order=["daytime", "night"])
ax.set(xlabel='phase', ylabel='count')
ax.set(ylim=(0,200000))

In [ ]:
#Memory savings
del train_taken_phase
del test_taken_phase
del train_taken_hour
del test_taken_hour
gc.collect()

### How many data are there per animal category Id?

There are a lot of categories in dataset. To confirme how many data are there in each categories, I plot barplot.  

In [ ]:
fig = plt.figure(figsize=(30, 4))
labels_id = sorted(list(set(df_categories["id"])))
ax = sns.barplot(x="id", y="count",data=df_categories, order=labels_id)
ax.set(ylabel='count')
ax.set(ylim=(0,80000))
plt.title('distribution of count per id in train')

In [ ]:
fig = plt.figure(figsize=(30, 4))
labels_id = sorted(list(set(df_categories["id"])))
ax = sns.barplot(x="id", y="count",data=df_categories_test, order=labels_id)
ax.set(ylabel='count')
ax.set(ylim=(0,80000))
plt.title('distribution of count per id in test')

Frequency of data in each id is similar for train and test data.

In [ ]:
fig = plt.figure(figsize=(15, 9))
ax = sns.distplot(df_categories['count'][1:])
ax.set(ylim=(0,0.00005))
ax.set(xlabel='count')
plt.title('distribution of number of data per id zoomed')

Almost data is less than 5000 pictures. 

But we can find that the number of specific IDs is very large and the data is biased.

### How many data per location?

We are required to detect photographs taken at different locations, but how distribute are data  in perspect of  location?

In [ ]:
labels_location_train = sorted(list(set(df_images['location'])))
labels_location_test = sorted(list(set(df_images_test['location'])))
labels_location = labels_location_train + labels_location_test

In [ ]:
fig = plt.figure(figsize=(30, 4))
ax = sns.countplot(df_images['location'], order=labels_location)
ax.set(xlabel='location', ylabel='count')
plt.title('Count of train data per location')

In [ ]:
fig = plt.figure(figsize=(30, 4))
ax = sns.countplot(df_images_test['location'], order=labels_location)
ax.set(xlabel='location', ylabel='count')
plt.title('Count of test data per location')

Train data and test data seems be completelly taken in different locations.

Number of pictures are greatly differend by location.

### Are different animal kinds photographed in different places?

We have location data and animal category data, so we can investigate wheathrer different animal kinds are photographed in different places.

In [ ]:
loc_cat_df_test = pd.merge(df_images, df_annotations, left_on='id', right_on='image_id', how = "inner").loc[:,["location", "category_id", "image_id"]]
loc_cat_df_test.head()

In [ ]:
loc_cat_dict = {}
for loc in set(loc_cat_df_test["location"]):
    loc_cat_dict[loc] = list(set(loc_cat_df_test[loc_cat_df_test["location"] == loc]["category_id"]))   
loc_cat_matrix = np.zeros([loc_cat_df_test["location"].max()+1, loc_cat_df_test["category_id"].max()+1])
loc_cat_matrix.shape

In [ ]:
for loc in loc_cat_dict.keys():
    for cat in loc_cat_dict[loc]:
        loc_cat_matrix[loc, cat] = 1

In [ ]:
fig = plt.figure(figsize=(15, 15))
ax = sns.heatmap(loc_cat_matrix)
ax.set(xlabel='category', ylabel='location')
plt.title('Relation between animal categories and locations')

It's a bit hard to see, but columns are the categories of animals and rows are the locations. Since similar patterns appear in the vertical direction, it seems that similar species may appear even in different places.

### How many animals in each photograph sequence?

We can know how many animals each photograph sequence (continuous frame of camera trap from startup to taking) by annotations value of train_annotations_json. Count key seems represent number of animals appear in each photograph sequence. 

Three Pan troglodytes appear in folloing frames.

In [ ]:
df_annotations[2429:2440]

In [ ]:
fig = plt.figure(figsize=(25, 16))
#for i,im_path in enumerate(df_annotations[df_annotations.loc[:,"count"] == 3].loc[:,"image_id"][2429:2440]):
for i,im_path in enumerate(df_annotations.loc[:,"image_id"][2429:2440]):
    ax = fig.add_subplot(4, 4, i+1, xticks=[], yticks=[])
    im = Image.open("../input/iwildcam-2020-fgvc7/train/" + im_path + ".jpg")
    im = im.resize((480,270))
    plt.imshow(im)

Does the number of individuals that appear at one time vary depending on the animal species?

In [ ]:
c = collections.Counter(df_annotations.loc[:,"count"])
animals_in_pict = list(c.keys())
freq = list(c.values())
k = zip(animals_in_pict,freq)

In [ ]:
animals_in_pict_df = pd.DataFrame(sorted(k),columns=['num_of_animals','freq'])

In [ ]:
animals_in_pict_df

Only a limited number of animals are grouped together. 266 species are taken in train set, but only 14 species were photographed, with more than 10 animals at a time. It turns out that there is a relationship between the animal species and the number of individuals reflected.　

In [ ]:
species_morethan_10 = list(set(df_annotations[df_annotations.loc[:,"count"] >= 10].loc[:,"category_id"]))
df_categories[df_categories.loc[:,"id"].isin(species_morethan_10)].loc[:,["id", "name"]]

Species that shows more than 30 individuals at a time in a photo is  only a cow and a boar.

In [ ]:
species_morethan_30 = list(set(df_annotations[df_annotations.loc[:,"count"] >= 30].loc[:,"category_id"]))
df_categories[df_categories.loc[:,"id"].isin(species_morethan_30)].loc[:,["id", "name"]]

In [ ]:
df_annotations[(df_annotations.loc[:,"category_id"] == 2) & (df_annotations.loc[:,"count"] >= 30)].head()

In [ ]:
fig = plt.figure(figsize=(25, 16))
#for i,im_path in enumerate(df_annotations[df_annotations.loc[:,"count"] == 3].loc[:,"image_id"][2429:2440]):
for i,im_path in enumerate(df_annotations.loc[:,"image_id"][149674:149678]):
    ax = fig.add_subplot(4, 4, i+1, xticks=[], yticks=[])
    im = Image.open("../input/iwildcam-2020-fgvc7/train/" + im_path + ".jpg")
    im = im.resize((480,270))
    plt.imshow(im)

In [ ]:
df_annotations[(df_annotations.loc[:,"category_id"] == 71) & (df_annotations.loc[:,"count"] >= 30)].head()

In [ ]:
jpeg ='../input/iwildcam-2020-fgvc7/train/8897a9f8-21bc-11ea-a13a-137349068a90.jpg'
im = Image.open(jpeg)
im = im.resize((480,270))
plt.imshow(im)

# Model example

I prepare EfficientNet example. The point of this model is that I choose photos which one or less　animal is taken for training.

I refered following kernels.

https://www.kaggle.com/ateplyuk/inat2019-starter-keras-efficientnet/data

https://www.kaggle.com/mobassir/keras-efficientnetb2-for-classifying-cloud

In [ ]:
!pip install tensorflow-gpu==1.14.0
!pip install keras==2.2.4
!pip install git+https://github.com/qubvel/efficientnet

import cv2
from copy import deepcopy
import efficientnet.keras as efn 
import glob
from IPython.display import Image
import json
import keras
import keras.backend as K
from keras.optimizers import Adam
from keras.applications.densenet import DenseNet201
from keras.layers import Dense, Flatten, Activation, Dropout, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers, applications
from keras.models import Model, load_model
from keras.callbacks import Callback, ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.utils import Sequence
import matplotlib.pyplot as plt
import multiprocessing
import numpy as np
from numpy.random import seed
import os
import random
from sklearn.metrics import precision_recall_curve, auc
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook as tqdm

seed(10)

%matplotlib inline

In [ ]:
train_anns_df = df_annotations[['image_id','category_id']]
train_img_df = df_images[['id', 'file_name']].rename(columns={'id':'image_id'})
df_train_file_cat = pd.merge(train_img_df, train_anns_df, on='image_id')
df_train_file_cat['category_id']=df_train_file_cat['category_id'].astype(str)
df_train_file_cat.head()

In [ ]:
#megadetector_results_df was done the following operation in the previous cell.
#megadetector_results_df = megadetector_results_df[megadetector_results_df['detected_num'] < 2]

df_train_file_cat = pd.merge(megadetector_results_df, df_train_file_cat, on='image_id', how='inner')

fig = plt.figure(figsize=(15, 4))
ax = sns.countplot(x="detected_num", data=megadetector_results_df)
ax.set(ylabel='count')
#ax.set(ylim=(0,80000))
plt.title('distribution of count per animals each data of train')

### Prepare data and model

In [ ]:
#Parameters

batch_size = 256
img_size = 96
lr = 0.001 
nb_classes = 267
nb_epochs = 6

In [ ]:
%%time

train_datagen=ImageDataGenerator(rescale=1./255, 
    validation_split=0.25,
    horizontal_flip = True,    
    zoom_range = 0.3,
    width_shift_range = 0.3,
    height_shift_range=0.3
    )

train_generator=train_datagen.flow_from_dataframe(    
    dataframe=df_train_file_cat[:50000],    
    directory="../input/iwildcam-2020-fgvc7/train",
    x_col="file_name",
    y_col="category_id",
    batch_size=batch_size,
    shuffle=True,
    classes = [ str(i) for i in range(nb_classes)],
    class_mode="categorical",    
    target_size=(img_size,img_size))

test_datagen = ImageDataGenerator(rescale=1./255)

valid_generator=test_datagen.flow_from_dataframe(    
    dataframe=df_train_file_cat[50000:],    
    directory="../input/iwildcam-2020-fgvc7/train",
    x_col="file_name",
    y_col="category_id",
    batch_size=batch_size,
    shuffle=True,
    classes = [ str(i) for i in range(nb_classes)],
    class_mode="categorical",  
    target_size=(img_size,img_size))

In [ ]:
def get_model():
    K.clear_session()
    base_model =  efn.EfficientNetB3(weights='imagenet', include_top=False, pooling='avg', input_shape=(img_size, img_size, 3))
    x = base_model.output
    predictions = Dense(nb_classes, activation="softmax")(x)
    return Model(inputs=base_model.input, outputs=predictions)

model = get_model()
model.compile(optimizers.Adam(lr=lr, decay=1e-6),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
early = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=1, mode='auto')

### Training

In [ ]:
%%time
history = model.fit_generator(generator=train_generator,  
                                    steps_per_epoch=5,
                                    validation_data=valid_generator, 
                                    validation_steps=2,
                                    epochs=nb_epochs,
                                    callbacks = [early],
                                    verbose=2)

In [ ]:
history_df = pd.DataFrame(history.history)
history_df[['loss', 'val_loss']].plot()
history_df[['acc', 'val_acc']].plot()

In [ ]:
import gc
del train_datagen, train_generator
gc.collect()

### Prediction

In [ ]:
sam_sub_df = pd.read_csv('../input/iwildcam-2020-fgvc7/sample_submission.csv')
sam_sub_df["file_name"] = sam_sub_df["Id"].map(lambda str : str + ".jpg")
sam_sub_df.head()

In [ ]:
%%time

test_generator = test_datagen.flow_from_dataframe(      
    
        dataframe=sam_sub_df,    
    
        directory = "../input/iwildcam-2020-fgvc7/test",    
        x_col="file_name",
        target_size = (img_size,img_size),
        batch_size = 1,
        classes = [ str(i) for i in range(nb_classes)],
        shuffle = False,
        class_mode = None
        )

In [ ]:
%%time
test_generator.reset()
predict=model.predict_generator(test_generator, steps = len(test_generator.filenames))
predicted_class_indices=np.argmax(predict,axis=1)
sam_sub_df["Category"] = predicted_class_indices
sam_sub_df = sam_sub_df.loc[:,["Id", "Category"]]
sam_sub_df.to_csv("submission.csv",index=False)

The orginal training set contains pictures of many animals at one time. I thought they have a bad impact to learning. By eliminating these, it seems that learning has converged well. Instead, the predictions are not very good. As one of the improvement plans I think, it may be effective to cut out the part of data where the animal is reflected.

## [WIP] Appendix

### About Landsat8

We are provided Landsat-8 multispectral imagery.

The Landsat 8 satellite takes pictures of the Earth using different wavelengths of light. Each of these are called 'bands'.  We reassemble them to tell us understand about Earth.

Features of Landsat8:

- The satellite circle the globe every 99 minutes.
- The satellite takes 16 days to passed over every spot.
- recorded data in 11 different wavelength.

Refer following videos!


In [ ]:
YouTubeVideo('mqVKR9OnqqA')

In [ ]:
YouTubeVideo('A6WzAc1FTeA')

※Note that the data for second video was collected by Landsat 5 on November 10, 2011.

We can get Landsat-8 multispectral imagery in officially announced following URL.

https://github.com/visipedia/iwildcam_comp#Data

We can get three type of files:

- multispectral.npy
- pixelqa.npy
- radsatqa.npy

Dataset seems include multispectral imageryies for each location and certain period. For example, there are data between 2013/6/11 to 2019/12/5 for location 003.

 Thank you for your reading!